In [1]:
import numpy
import sympy
from matplotlib import pyplot
sympy.init_printing()

$$
\begin{eqnarray}
\rho &=& \rho (x, t) \\
v &=& v_{max} (1 - \rho / \rho_{max}) \\
F &=& v \rho  \\
F &=& v_{max} \rho (1 - \rho / \rho_{max})  \\
\frac{\partial \rho}{\partial t} + \frac{\partial F}{\partial x} &=& 0 \\
\frac{\partial \rho}{\partial t} + \frac{\partial (v_{max} \rho [1 - \rho / \rho_{max}])}{\partial x} &=& 0 \\
\frac{\partial \rho}{\partial t} + v_{max} \frac{\partial (\rho - {\rho}^2 / \rho_{max})}{\partial x} &=& 0  \\
\frac{\partial \rho}{\partial t} + v_{max} \left( \frac{\partial \rho}{\partial x} - \frac{2 \rho}{\rho_{max}} \frac{\partial \rho}{\partial x} \right) &=& 0 \\
\frac{\partial \rho}{\partial t} + v_{max} \left( 1 - \frac{2 \rho}{\rho_{max}} \right) \frac{\partial \rho}{\partial x} &=& 0 \\
\end{eqnarray}
$$
<br>
<br>
Make traffic flow equation discrete using forward difference in time and backward difference
in space.
<br>
<br>

$$
\begin{eqnarray}
\frac{\rho_i^{n+1} - \rho_i^n}{\Delta t} &=& -v_{max} \left( 1 - \frac{2 \rho_i^n}{\rho_{max}} \right) \left( \frac{\rho_i^n - \rho_{i-1}^n}{\Delta x}\right) \\
\rho_i^{n+1} &=& \rho_i^n -v_{max} \frac{\Delta t}{\Delta x} \left( 1 - \frac{2 \rho_i^n}{\rho_{max}} \right) \left( \rho_i^n - \rho_{i-1}^n \right)
\end{eqnarray}
$$

In [3]:
v_max = 80.0  # Maximum vehicle speed (km/hr)
L = 11.0  # Length of road (km)
𝜌_max = 250  # Maximum number density of cars per kilometer (cars/km)
nx = 51  # Number of spatial grid points
Δx = L / (nx - 1)  # spatial grid size
nt = 51  # Number of time steps
Δt = 0.001  # time step size (hr)

## Simulation results:  Part A

In [4]:
# Set initial conditions for traffic density
x = numpy.linspace(0, L, nx)
𝜌0 = numpy.ones(nx) * 10
𝜌0[10:20] = 50
print(x)
print(𝜌0)

[ 0.    0.22  0.44  0.66  0.88  1.1   1.32  1.54  1.76  1.98  2.2   2.42
  2.64  2.86  3.08  3.3   3.52  3.74  3.96  4.18  4.4   4.62  4.84  5.06
  5.28  5.5   5.72  5.94  6.16  6.38  6.6   6.82  7.04  7.26  7.48  7.7
  7.92  8.14  8.36  8.58  8.8   9.02  9.24  9.46  9.68  9.9  10.12 10.34
 10.56 10.78 11.  ]
[10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 50. 50. 50. 50. 50. 50. 50. 50.
 50. 50. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10.
 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10.]


Boundary condition:  $\rho(0, t) = 10$

In [34]:
# Compute the number density solution using Euler's method and array slicing
𝜌 = 𝜌0.copy()
for n in range(1, nt):
    𝜌[1:] = 𝜌[1:] - v_max * Δt / Δx * (1 - 2 * 𝜌[1:] / 𝜌_max) * (𝜌[1:] - 𝜌[:-1])
print(n, 𝜌)

50 [10.         10.         10.         10.         10.         10.
 10.         10.         10.         10.         10.00000011 10.00000291
 10.00003893 10.00034269 10.00223898 10.01159783 10.04969404 10.18139155
 10.57562616 11.60605225 13.93829489 18.36940784 25.08804849 32.85174524
 39.60312372 44.08292131 46.26050424 46.65659328 45.78842525 44.02311368
 41.61049542 38.73600094 35.55286007 32.19717659 28.79455135 25.46288236
 22.31250635 19.44331313 16.93844467 14.85535362 13.21691364 12.00682836
 11.17310501 10.6400742  10.32502848 10.15328441 10.06703446 10.02715165
 10.01017436 10.00352303 10.00112572]


Find the *minimum* velocity at time t = 0 in <u>meters per second</u>. Round to 2 decimal places.

Velocity is a minimum where traffic density is highest, and at t = 0 (initial condition), the highest traffic density is
$ \rho = 50 $ cars/km.

In [18]:
𝜌 = 50
v = v_max * (1 - 𝜌 / 𝜌_max)
v_min = v * 1000 / 3600  # km/hr * m/km * hr/s = m/s
print("Answer:")
print("-------")
print("Minimum velocity at t = 0 is {:.2f} m/s".format(v_min))

Answer:
-------
Minimum velocity at t = 0 is 17.78 m/s


Find the average velocity at time t = 3 minutes in <u>meters per second</u>. Round to 2 decimal places.

In [34]:
(50 * 0.001) * 60 # hr * min/hr = min

To solve, compute $\rho$ and $v$ at each space grid point for index 50 (50 time-steps = 3 min). Add the velocities from each space grid point at time index 50 and divide by the total number of grid points to obtain the average velocity.

In [29]:
# Compute the velocity solution
v = numpy.ones(nx)
for i in range(nx):
    v[i] = v_max * (1 - 𝜌[i] / 𝜌_max) * 1000 / 3600  # km/hr * m/km * hr/s = m/s
    print(i, 𝜌[i], v[i])

0 10.0 21.333333333333332
1 10.0 21.333333333333332
2 10.0 21.333333333333332
3 10.0 21.333333333333332
4 10.0 21.333333333333332
5 10.0 21.333333333333332
6 10.0 21.333333333333332
7 10.0 21.333333333333332
8 10.0 21.333333333333332
9 10.0 21.333333333333332
10 10.000000107755067 21.333333323755106
11 10.000002914676662 21.333333074250962
12 10.000038931832615 21.33332987272599
13 10.000342694528292 21.33330287159749
14 10.002238975100529 21.333134313324397
15 10.011597834224807 21.332302414735572
16 10.049694044357626 21.32891608494599
17 10.181391550652629 21.317209639941993
18 10.575626157628255 21.28216656376638
19 11.606052253399554 21.19057313303115
20 13.938294889763174 20.983262676465493
21 18.369407841228863 20.589385969668548
22 25.088048486736803 19.992173467845618
23 32.85174523730992 19.30206709001689
24 39.60312371853437 18.7019445583525
25 44.08292131053428 18.303740327952507
26 46.26050423739899 18.11017740112009
27 46.65659327688548 18.074969486499068
28 45.7884252537

In [32]:
# Compute average velocity
average = sum(v) / nx
print("Answer:")
print("-------")
print("Average velocity at t = 3 min is {:.2f} m/s".format(average))

Answer:
-------
Average velocity at t = 3 min is 20.56 m/s


Find the *minimum* velocity at time t = 6 minutes in <u>meters per second</u>. Round to 2 decimal places.

In [33]:
(100 * 0.001) * 60 # hr * min/hr = min

In [35]:
nt = 101  # Number of time steps

In [36]:
# Compute the number density solution using Euler's method and array slicing
𝜌 = 𝜌0.copy()
for n in range(1, nt):
    𝜌[1:] = 𝜌[1:] - v_max * Δt / Δx * (1 - 2 * 𝜌[1:] / 𝜌_max) * (𝜌[1:] - 𝜌[:-1])
print(n, 𝜌)

100 [10.         10.         10.         10.         10.         10.
 10.         10.         10.         10.         10.         10.
 10.         10.         10.         10.         10.         10.00000003
 10.00000017 10.00000092 10.00000452 10.00001993 10.00008016 10.00029616
 10.0010118  10.00321528 10.00955475 10.02668136 10.07031726 10.1755223
 10.41592939 10.93572232 11.99095581 13.96785925 17.28460046 22.06845034
 27.76384442 33.22308337 37.4092647  39.93350548 40.964339   40.87964847
 40.03633888 38.69884287 37.04509708 35.1917461  33.21663789 31.17434568
 29.10587576 27.04448252 25.01906055]


In [37]:
# Compute the velocity solution
v = numpy.ones(nx)
for i in range(nx):
    v[i] = v_max * (1 - 𝜌[i] / 𝜌_max) * 1000 / 3600  # km/hr * m/km * hr/s = m/s
    print(i, 𝜌[i], v[i])

0 10.0 21.333333333333332
1 10.0 21.333333333333332
2 10.0 21.333333333333332
3 10.0 21.333333333333332
4 10.0 21.333333333333332
5 10.0 21.333333333333332
6 10.0 21.333333333333332
7 10.0 21.333333333333332
8 10.0 21.333333333333332
9 10.0 21.333333333333332
10 10.000000000000002 21.333333333333332
11 10.000000000000009 21.333333333333332
12 10.000000000000208 21.333333333333318
13 10.000000000003567 21.33333333333302
14 10.000000000045464 21.333333333329293
15 10.000000000459858 21.333333333292458
16 10.000000003847111 21.333333332991366
17 10.000000027385688 21.33333333089905
18 10.000000169375575 21.333333318277724
19 10.000000924871516 21.333333251122532
20 10.000004516082502 21.33333293190378
21 10.000019926830385 21.333331562059524
22 10.00008015506628 21.333326208438553
23 10.000296158063202 21.33330700817216
24 10.001011802652975 21.333243395319737
25 10.003215281841712 21.33304753050296
26 10.00955475353522 21.33248402190798
27 10.026681355127117 21.33096165732203
28 10.07031

In [44]:
# Find minimum velocity
minimum = min(v)
print("Answer:")
print("-------")
print("Minimum velocity at t = 6 min is {:.2f} m/s".format(minimum))

Answer:
-------
Minimum velocity at t = 6 min is 18.58 m/s


## Simulation Results:  Part B

Now, set $v_{max} = 136$ m/s and repeat the simulation using the following initial conditions for traffic density:

In [45]:
x = numpy.linspace(0, L, nx)
𝜌0 = numpy.ones(nx) * 20  #note this change
𝜌0[10:20] = 50
print(𝜌0)

[20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 50. 50. 50. 50. 50. 50. 50. 50.
 50. 50. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20.
 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20.]


In [46]:
v_max = 136.0  # Maximum vehicle speed (km/hr)
L = 11.0  # Length of road (km)
𝜌_max = 250  # Maximum number density of cars per kilometer (cars/km)
nx = 51  # Number of spatial grid points
Δx = L / (nx - 1)  # spatial grid size
nt = 51  # Number of time steps
Δt = 0.001  # time step size (hr)

Boundary condition:  $\rho(0, t) = 20$

Find the *minimum* velocity at time t = 0 in meters per second. Round to 2 decimal places.

Velocity is a minimum where traffic density is highest, and at t = 0 (initial condition), the highest traffic density is  𝜌=50  cars/km.

In [47]:
𝜌 = 50
v = v_max * (1 - 𝜌 / 𝜌_max)
v_min = v * 1000 / 3600  # km/hr * m/km * hr/s = m/s
print("Answer:")
print("-------")
print("Minimum velocity at t = 0 is {:.2f} m/s".format(v_min))

Answer:
-------
Minimum velocity at t = 0 is 30.22 m/s


Find the average velocity at time t = 3 minutes in meters per second. Round to 2 decimal places.

In [48]:
time = (50 * 0.001) * 60 # hr * min/hr = min
time

To solve, compute $\rho$ and $v$ at each space grid point for index 50 (50 time-steps = 3 min). Add the velocities from each space grid point at time index 50 and divide by the total number of grid points to obtain the average velocity.

In [49]:
# Compute the number density solution using Euler's method and array slicing
𝜌 = 𝜌0.copy()
for n in range(1, nt):
    𝜌[1:] = 𝜌[1:] - v_max * Δt / Δx * (1 - 2 * 𝜌[1:] / 𝜌_max) * (𝜌[1:] - 𝜌[:-1])
print(n, 𝜌)

50 [20.         20.         20.         20.         20.         20.
 20.         20.         20.         20.         20.         20.
 20.         20.         20.         20.00000003 20.00000024 20.00000174
 20.00001077 20.00005832 20.0002796  20.0011994  20.00464543 20.01637475
 20.0528934  20.15744865 20.43333582 21.10135681 22.56065568 25.3341569
 29.67886442 35.00494992 40.00032278 43.60953513 45.58633475 46.21340229
 45.88213099 44.90335403 43.48518166 41.76283846 39.8292146  37.7549989
 35.60012704 33.4199835  31.26868004 29.20045876 27.26938253 25.52702401
 24.01791329 22.77318579 21.80411639]


In [50]:
# Compute the velocity solution
v = numpy.ones(nx)
for i in range(nx):
    v[i] = v_max * (1 - 𝜌[i] / 𝜌_max) * 1000 / 3600  # km/hr * m/km * hr/s = m/s
    print(i, 𝜌[i], v[i])

0 20.0 34.75555555555555
1 20.0 34.75555555555555
2 20.0 34.75555555555555
3 20.0 34.75555555555555
4 20.0 34.75555555555555
5 20.0 34.75555555555555
6 20.0 34.75555555555555
7 20.0 34.75555555555555
8 20.0 34.75555555555555
9 20.0 34.75555555555555
10 20.000000000000007 34.75555555555555
11 20.00000000000041 34.755555555555496
12 20.000000000011244 34.755555555553855
13 20.000000000202906 34.75555555552489
14 20.00000000269735 34.75555555514795
15 20.00000002818425 34.7555555512966
16 20.000000241129257 34.755555519118246
17 20.00000173759944 34.75555529298497
18 20.00001076828463 34.7555539283481
19 20.000058319566094 34.755546742821124
20 20.000279603416832 34.75551330437257
21 20.001199396425502 34.75537431342904
22 20.004645432456847 34.75485357909541
23 20.016374745866568 34.7530811495135
24 20.052893404512627 34.7475627744292
25 20.157448653676425 34.731763314555565
26 20.43333581848057 34.690073698540715
27 21.101356806785848 34.589128304752364
28 22.560655677439648 34.36861203

In [52]:
# Compute average velocity
average = sum(v) / nx
print("Answer:")
print("-------")
print("Average velocity at t = 3 min is {:.2f} m/s".format(average))

Answer:
-------
Average velocity at t = 3 min is 33.77 m/s


Find the *minimum* velocity at time <u>t = 3</u> minutes in meters per second. Round to 2 decimal places.

In [55]:
# Find minimum velocity
minimum = min(v)
print("Answer:")
print("-------")
print("Minimum velocity at t = 3 min is {:.2f} m/s".format(minimum))

Answer:
-------
Minimum velocity at t = 3 min is 30.79 m/s
